In [1]:
# imports

import pandas as pd
import re
import emoji
from nltk.corpus import stopwords

import utils

### Reading Files

In [2]:
train_set = pd.read_csv("./train.csv")
dev_set = pd.read_csv("./dev.csv")

### Cleaning Tweets

In [3]:
cleaned_train_set = train_set.copy()
cleaned_dev_set = dev_set.copy()
cleaned_train_set['text'] = cleaned_train_set['text'].apply(utils.clean_tweet)
cleaned_dev_set['text'] = cleaned_dev_set['text'].apply(utils.clean_tweet)

### Duplicated Rows

In [4]:
# check duplicated rows
train_duplicated_rows = train_set[train_set.duplicated(subset=['text'], keep=False)]
dev_duplicated_rows = dev_set[dev_set.duplicated(subset=['text'], keep=False)]


### Stop Words

In [5]:
stopwords_list = stopwords.words('arabic')
print(stopwords_list)


['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

## Investigating Cleaning Steps

In [6]:
processed_train_set = train_set.copy()
processed_dev_set = dev_set.copy()

# remove <LF>
processed_train_set['text'] = train_set['text'].str.replace(
    "<LF>", " ", regex=True)
processed_dev_set['text'] = processed_dev_set['text'].str.replace(
    "<LF>", " ", regex=True)
# remove URLs
processed_train_set['text'] = processed_train_set['text'].str.replace(
    r'(https?://\S+)', "", regex=True)
processed_dev_set['text'] = processed_dev_set['text'].str.replace(
    r'(https?://\S+)', "", regex=True)

# remove mentions
processed_train_set['text'] = processed_train_set['text'].str.replace(r"@\w*", "", regex=True)
processed_dev_set['text'] = processed_dev_set['text'].str.replace(r"@\w*", "", regex=True)

In [7]:
# check tweets with english characters

train_tweets_with_english = processed_train_set[processed_train_set['text'].str.contains(r'[a-zA-Z]')]
dev_tweets_with_english = processed_dev_set[processed_dev_set['text'].str.contains(r'[a-zA-Z]')]

In [8]:
# check tweets with abbreviations

train_tweets_with_dots = processed_train_set[processed_train_set['text'].str.contains(r'(?:[^\.\s]\.){2,}')]
dev_tweets_with_dots = processed_dev_set[processed_dev_set['text'].str.contains(r'(?:[^\.\s]\.){2,}')]

# joined_characters_train = processed_train_set['text'].str.replace(
#     r'(?:\S\.){2,}', lambda string: re.sub(r'\.', "", string))


In [9]:
# check tweets with emojis
def filter_emojis(string):
    emojis = emoji.distinct_emoji_list(string)
    for emoji_char in emojis:
        string =  re.sub(emoji_char, "", string)
    
    return string

processed_train_set['text'] =  processed_train_set['text'].apply(filter_emojis)
processed_dev_set['text'] = processed_dev_set['text'].apply(filter_emojis)